create_agent会自动处理结构化输出，只需设定所需的结构化输出schema，当模型生成结构化数据时，该数据会被自动捕获、验证，并存入智能体状态中的`structured_response`键中。

In [7]:
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(model="deepseek-chat", temperature=0.7)

# 结构化参数

结构化支持工具调用或者模型内置的格式化两种策略：
- ToolStrategy[StructuredResponseT]：通过工具调用（tool calling）实现结构化输出
- ProviderStrategy[StructuredResponseT]：使用模型提供商原生的结构化输出功能
- type[StructuredResponseT]：直接指定结构化数据的 Schema 类型 —— LangChain 会根据模型能力自动选择最优策略
- None：不启用结构化输出

当直接提供一个 Schema 类型时，LangChain 会自动选择：
- 对于支持原生结构化输出的模型（例如 OpenAI、Anthropic 或 Grok），使用 ProviderStrategy；（如果使用 langchain >= 1.1，对模型原生结构化输出功能的支持会从模型的配置文件（profile data）中动态读取。）
- 对于其他模型，则使用 ToolStrategy。

## Provider strategy

某些模型提供商（例如 OpenAI、Grok、Gemini）通过其 API 原生支持结构化输出。在可用的情况下，这是最可靠的方法。通过配置ProviderStrategy即可启用此策略：

In [3]:
from typing import Generic, TypeVar

SchemaT = TypeVar("SchemaT")

class ProviderStrategy(Generic[SchemaT]):
    schema: type[SchemaT]

## Tool calling strategy

对于不支持原生结构化输出的模型，LangChain 会通过工具调用（tool calling）来实现相同的效果。该方法适用于所有支持工具调用的模型，而大多数现代模型都具备这一能力。

In [5]:
from typing import Callable, Union

class ToolStrategy(Generic[SchemaT]):
    schema: type[SchemaT] # required，支持Union types
    tool_message_content: str | None # 用于在生成结构化输出时返回的工具消息的自定义内容。如果未提供，则默认使用一条显示结构化响应数据的消息。
    handle_errors: Union[
        bool,
        str,
        type[Exception],
        tuple[type[Exception], ...],
        Callable[[Exception], str],
    ]

In [ ]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.tools import tool
from langchain.agents.structured_output import ToolStrategy, ProviderStrategy

@tool
def get_weather(location: str) -> str:
    """获取指定位置的天气信息"""
    return f"{location}的天气是晴天，气温25℃，湿度50%"


class WeatherInfo(BaseModel):
    location: str
    weather: str

agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="你是一个乐于助人的助手",
    response_format=ToolStrategy(WeatherInfo)
    # 或者仅仅指定schema，让LangChain来自动匹配
    # response_format=WeatherInfo
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "宁波天气如何"}]
})

print(result["structured_response"])


location='宁波' weather='晴天，气温25℃，湿度50%'


# 自定义格式化工具消息

> tool_message_content只会改变格式化工具的输出，结构化的输出不会丢失，存储在structured_response中。

In [12]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain_deepseek import ChatDeepSeek
from pydantic import BaseModel, Field
from typing import Literal

# 初始化 DeepSeek 模型
model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.7
)

# 定义会议行动项的数据模型
class MeetingAction(BaseModel):
    """从会议记录中提取的行动项。"""
    task: str = Field(description="需要完成的具体任务")
    assignee: str = Field(description="负责该任务的人员")
    priority: Literal["low", "medium", "high"] = Field(description="优先级")

# 创建智能体
agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=MeetingAction,
        tool_message_content="已捕获行动项并添加到会议纪要中！"
    )
)

# 调用智能体
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "根据我们的会议内容：Sarah 需要尽快更新项目时间表。"}]
})

# 打印输出结果
for message in result["messages"]:
    message.pretty_print()

# 原本的结构化响应
print("\n\n原本的结构化响应:", result["structured_response"])


================================ Human Message =================================

根据我们的会议内容：Sarah 需要尽快更新项目时间表。
================================== Ai Message ==================================
Tool Calls:
  MeetingAction (call_00_Qd7ks2ANSd1OEMnGblq4xXMm)
 Call ID: call_00_Qd7ks2ANSd1OEMnGblq4xXMm
  Args:
    task: 更新项目时间表
    assignee: Sarah
    priority: high
================================= Tool Message =================================
Name: MeetingAction

已捕获行动项并添加到会议纪要中！


原本的结构化响应: task='更新项目时间表' assignee='Sarah' priority='high'


tool_message_content参数定义了当模型成功提取数据后，系统自动返回给用户的确认消息（“已捕获行动项...”），提升了交互体验。

# 错误处理

模型在通过工具调用生成结构化输出时可能会出错。LangChain提供了智能的重试机制，可自动处理这些错误，默认值为 True。
- True：捕获所有错误，并使用默认的错误模板进行处理。
- str：捕获所有错误，并使用此自定义消息进行处理。
- type[Exception]：仅捕获指定类型的异常，并使用默认错误消息。
- tuple[type[Exception], ...]：仅捕获元组中列出的异常类型，并使用默认错误消息。
- Callable[[Exception], str]：提供一个自定义函数，该函数接收异常对象并返回错误消息。
- False：不进行重试，直接让异常向上抛出（不捕获）。

## 调用了多个格式化shema

In [34]:
from pydantic import BaseModel, Field
from typing import Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.7
)

class ContactInfo(BaseModel):
    name: str = Field(description="Person's name")
    email: str = Field(description="Email address")

class EventDetails(BaseModel):
    event_name: str = Field(description="Name of the event")
    date: str = Field(description="Event date")

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(Union[ContactInfo, EventDetails])  
    # Default: handle_errors=True
)

for event in agent.stream({
    "messages": [{"role": "user", "content": "Extract info: John Doe (john@email.com) is organizing Tech Conference on March 15th"}],
},
    stream_mode="updates"):
    for step, data in event.items():
        print(f"step: {step}")
        data['messages'][-1].pretty_print()


step: model
================================= Tool Message =================================
Name: EventDetails

Error: Model incorrectly returned multiple structured responses (ContactInfo, EventDetails) when only one is expected.
 Please fix your mistakes.
step: model
================================= Tool Message =================================
Name: ContactInfo

Returning structured response: name='John Doe' email='john@email.com'


## Schema验证错误

In [29]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain_deepseek import ChatDeepSeek

# 初始化 DeepSeek 模型
model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.7
)

# 定义产品评分的数据模型
class ProductRating(BaseModel):
    rating: int | None = Field(description="评分，范围1-5", ge=1, le=5)
    comment: str = Field(description="评价内容")

# 创建智能体
agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(ProductRating),  
    # 默认: handle_errors=True
    system_prompt="你是一个用于解析产品评论的助手。请根据用户输入提取评分和评论。不要编造任何字段或值。"
)

# 打印输出结果
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

解析这段内容：Amazing product, 10/10!
================================== Ai Message ==================================
Tool Calls:
  ProductRating (call_00_1mcRdJxBU7bv56ZubTnnRPrU)
 Call ID: call_00_1mcRdJxBU7bv56ZubTnnRPrU
  Args:
    rating: 5
    comment: Amazing product, 10/10!
================================= Tool Message =================================
Name: ProductRating

Returning structured response: rating=5 comment='Amazing product, 10/10!'


模型无法给出确切的 1-5 分（用户说的是 10/10），根据 handle_errors=True 的默认设置，模型可能会将 rating 设为 None 或尝试推理（取决于模型理解），但一定会返回一个包含 comment 字段的结构化响应。

这里deepseek会自己调整到最高分数5分，没有办法演示，因此这里引用官方的输出结果如下：

## str类型

如果handle_errors是一个字符串，智能体将始终使用固定的工具消息提示模型重新尝试，也就是说当你把 handle_errors参数设为一个字符串（string）时，LangChain 会用这个字符串作为固定的错误提示消息，并通过 ToolMessage 告诉模型：“你出错了，请重试”，而且每次出错都用同样的这句话，不会根据具体错误类型或内容动态调整。

In [ ]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain_deepseek import ChatDeepSeek
from pydantic import BaseModel, Field
from typing import Literal

# 初始化 DeepSeek 模型
model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.7
)

# 定义产品评分的数据模型
class ProductRating(BaseModel):
    rating: int | None = Field(description="评分，范围1-5", ge=1, le=5)
    comment: str = Field(description="评价内容")

# 创建智能体
agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        # 错误处理：当模型输出格式不符合要求时，返回自定义提示
        ProductRating,
        handle_errors="请提供 1 到 5 之间的有效评分，并包含评论内容。"
    ),
    system_prompt="你是一个用于解析产品评论的助手。请根据用户输入提取评分和评论。不要编造任何字段或值。"
)


for event in agent.stream({
    "messages": [{"role": "user", "content": "解析这段内容：Amazing product, 10/10!"}]
},
    stream_mode="updates"):
    for step, data in event.items():
        print(f"step: {step}")
        data['messages'][-1].pretty_print()

## 捕获特定的错误

如果 handle_errors 是一个异常类型，智能体仅在抛出的异常属于该指定类型时才会进行重试（使用默认的错误消息）；在所有其他情况下，异常将直接抛出，不会被拦截或重试。

单一错误类型：

In [ ]:
from langchain.agents.structured_output import MultipleStructuredOutputsError

ToolStrategy(
    schema=ProductRating,
    handle_errors=MultipleStructuredOutputsError  # Only retry on MultipleStructuredOutputsError, raise others
)

通过元祖指定多个类型：

In [ ]:
ToolStrategy(
    schema=ProductRating,
    handle_errors=(ValueError, TypeError)  # Retry on ValueError and TypeError
)

## 自定义错误处理函数

In [ ]:
from typing import Union

from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain_deepseek import ChatDeepSeek
from langchain.agents.structured_output import StructuredOutputValidationError
from langchain.agents.structured_output import MultipleStructuredOutputsError
from pydantic import BaseModel, Field

# 初始化 DeepSeek 模型
model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.7
)

# 自定义错误处理器
def custom_error_handler(error: Exception) -> str:
    if isinstance(error, StructuredOutputValidationError):
        return "格式有误。请重试。"
    elif isinstance(error, MultipleStructuredOutputsError):
        return "返回了多个结构化结果。请挑选其中最相关的一个。"
    else:
        return f"错误: {str(error)}"

# 定义联系人信息模型
class ContactInfo(BaseModel):
    name: str = Field(description="人名")
    email: str = Field(description="电子邮箱地址")

# 定义活动详情模型
class EventDetails(BaseModel):
    event_name: str = Field(description="活动名称")
    date: str = Field(description="活动日期")

# 创建智能体
agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=Union[ContactInfo, EventDetails],  # 支持联合类型
        # 报错：StructuredOutputValidationError
        # handle_errors=StructuredOutputValidationError
        
        # 捕捉并返回默认的错误信息
        # handle_errors=MultipleStructuredOutputsError
        handle_errors=custom_error_handler  # 使用自定义错误处理函数
    )
)

# 调用智能体
result = agent.invoke({
    "messages": [{"role": "user", "content": "提取信息：John Doe (john@email.com) 正在组织 3月15日 的 Tech Conference"}]
})

# 打印输出结果
for msg in result['messages']:
    msg.pretty_print()

# TIPS

如果tool输出的内容和response_format的格式不匹配，会出问题，可以传入union多个类来让agent自行选择：response_format=ToolStrategy(Union[ContactInfo, WeatherInfo])

response_format=ContactInfo这种写法，会自动选择ProviderStrategy这种模式，在功能上等同于写成 response_format=ProviderStrategy(ContactInfo)。提供商原生的结构化输出由于由模型提供商强制执行Schema，因此具有高可靠性和严格的验证机制。在可用时应优先使用。无论采用哪种写法，如果该模型实际上不支持结构化输出，智能体都会自动回退到工具调用（tool calling）策略。